### <center>2018 Winter CS101.07</center>

# <center>链表</center>

##### <center>by tanzhuxiaqiu@huawei.com</center>

## 回顾

- 动态数组
- 栈
- 队列

## 今日议程

1. 单向链表
2. 环形链表
3. 双向链表
4. 链表 v.s. 数组

## 单向链表

多个节点（Node）集合组成的一个线性序列。

#### 单向链表的节点

- 元素成员：表示节点存储的元素值
- 指针成员：指向单向链表中的后继节点，如果没有后继则为空

![](img/7-1.png)

- 头节点：链表中第一个节点
- 尾节点：链表中最后一个节点

![](img/7-2.png)

In [1]:
class SinglyLinkedListNode:
    __slots__ = ('_element', '_next')

    def __init__(self, e):
        self._element = e
        self._next = None

### 单向链表的几个基本操作


[visualgo](https://visualgo.net/en/list)

#### 在单向链表头部插入一个元素

![](img/7-6.png)

伪代码
```
Algorithm add_first(L, e):
    n = Node(e)
    n.next = L.head #1
    L.head = n #2
    L.size += 1
```

#### 在单向链表尾部插入一个元素

伪代码

![](img/7-7.png)

```
Algorithm add_last(L, e):
    n = Node(e)
    n.next = None
    L.tail.next = n #1
    L.tail = n #2
    L.size += 1
```

#### 在单向链表头部删除一个元素

伪代码

```
Algorithm del_first(L):
    if L.head in None then
        raise an error: the list is empty
    L.head = L.head.next
    L.size -= 1
```

#### 思考：如何在单向链表的尾部删除一个元素？

### 利用单向链表实现一个栈

- 把单向链表的头部作为栈顶

In [2]:
from abc import ABC, abstractmethod


class AbstractStack(ABC):
    """Abstract Class for Stacks."""
    def __init__(self):
        self._top = -1

    def __len__(self):
        return self._top + 1

    def __repr__(self):
        res = "->".join(map(str, self))
        return 'Top->' + res

    def is_empty(self):
        return self._top == -1

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError

    @abstractmethod
    def push(self, e):
        raise NotImplementedError

    @abstractmethod
    def pop(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError


In [3]:
class LinkedListStack(AbstractStack):

    def __init__(self):
        super().__init__()
        self._head = None

    def __iter__(self):
        p = self._head
        while p is not None:
            yield p._element
            p = p._next

    def push(self, e):
        node = SinglyLinkedListNode(e)
        node._next = self._head
        self._head = node
        self._top += 1

    def pop(self):
        if self.is_empty():
            raise IndexError('Stack is empty')
        res = self._head._element
        self._head = self._head._next
        self._top -= 1
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError('Stack is empty')
        return self._head._element

In [4]:
stack = LinkedListStack()
for i in range(5):
    stack.push(i)
stack

Top->4->3->2->1->0

In [5]:
print(stack.pop())
print(stack.pop())
print(stack.pop())
print(stack)

4
3
2
Top->1->0


#### 基于单向链表的栈的时间复杂度

|操作|示例|时间复杂度|
|---|---|---|---|
| Push | S.push(e) | O(1) |
| Pop | S.pop() | O(1) |
| Peek | S.peek() | O(1) |
| Is empty | S.is_empty() | O(1) |
| Length | len(S) | O(1) |

### 利用单向链表实现一个队列

- 需要维护头部和尾部两个指针
- 因为很难在单向链表尾部删除元素，所以出队列的操作应该放在单向链表的头部完成
- 入队在单向链表的尾部完成

In [6]:
from abc import ABC, abstractmethod


class AbstractQueue(ABC):

    def __init__(self):
        self._size = 0

    def __len__(self):
        return self._size

    def __repr__(self):
        return "->".join(map(str, self))

    def is_empty(self):
        return self._size == 0

    @abstractmethod
    def enqueue(self, e):
        raise NotImplementedError

    @abstractmethod
    def dequeue(self):
        raise NotImplementedError

    @abstractmethod
    def peek(self):
        raise NotImplementedError

    @abstractmethod
    def __iter__(self):
        raise NotImplementedError

In [7]:
class LinkedListQueue(AbstractQueue):

    def __init__(self):
        super().__init__()
        self._head = None
        self._tail = None

    def __iter__(self):
        p = self._head
        while p is not None:
            yield p._element
            p = p._next

    def enqueue(self, e):
        node = SinglyLinkedListNode(e)
        if self.is_empty():
            self._head = node
        else:
            self._tail._next = node
        self._tail = node
        self._size += 1

    def dequeue(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        res = self._head._element
        self._head = self._head._next
        self._size -= 1
        if self.is_empty():
            self._tail = None
        return res

    def peek(self):
        if self.is_empty():
            raise IndexError("Queue is empty")
        return self._head._element

In [8]:
queue = LinkedListQueue()
for i in range(5):
    queue.enqueue(i)
print(queue)

0->1->2->3->4


In [9]:
print(queue.dequeue())
print(queue.dequeue())
print(queue.dequeue())
print(queue)

0
1
2
3->4


## 环形链表

### Circular Linked List

- 将链表尾部的指针指向链表自己的头部
- 相比利用数组和取模运算实现的环形结构，环形链表实现了物理上的闭环、

![](img/7-3.png)

### 利用环形链表实现环形队列

- 只需要一个尾指针即可完成队列的基本操作
- 新增一个rotate()方法实现队列的轮转（头部第一个元素排到队列的末尾，也可以理解为第一个元素出列后再入列）

In [10]:
class CircularQueue(AbstractQueue):
    def __init__(self):
        super().__init__()
        self._tail = None

    def __iter__(self):
        p = self._tail._next
        while True:
            yield p._element
            if p is self._tail:
                return
            p = p._next

    def enqueue(self, e):
        node = SinglyLinkedListNode(e)
        if self.is_empty():
            node._next = node
        else:
            node._next = self._tail._next
            self._tail._next = node
        self._tail = node
        self._size += 1

    def dequeue(self):
        remove_node = self._tail._next
        if len(self) == 1:
            self._tail = None
        else:
            self._tail._next = remove_node._next
        self._size -= 1
        return remove_node._element

    def peek(self):
        return self._tail._next._element

    def rotate(self):
        if len(self) > 0:
            self._tail = self._tail._next

In [11]:
queue = CircularQueue()
for i in range(5):
    queue.enqueue(i)
print(queue)
print(queue.dequeue())
print(queue.dequeue())

0->1->2->3->4
0
1


In [12]:
print(queue)
queue.rotate()
print(queue)

2->3->4
3->4->2


## 双向链表

因为单向链表的不对称性带来了一些缺陷：

- 因为不能定位前驱节点，难以删除链表中除头节点外的任意节点
- 只能单向遍历链表

#### Double Linked List
如果一个链表中每个节点（Node）都同时维护了指向其前驱节点和后继节点的引用，我们称这种链表为双向链表。

#### 双向链表的节点

- 元素成员：表示节点存储的元素值
- 前驱指针成员：指向链表中的前驱节点，如果没有前驱则为空
- 后继指针成员：指向链表中的后继节点，如果没有后继则为空
- 哨兵节点：使链表在头尾的插入和删除操作更简便

![](img/7-4.png)

In [13]:
class DoubleLinkedListNode:
    __slots__ = ('_element', '_prev', '_next')

    def __init__(self, e):
        self._element = e
        self._prev = None
        self._next = None

### 利用双向链表实现双端队列

- 设置哨兵节点
- 将插入和删除元素的操作统一化

In [14]:
class DoubleLinkedList:
    class _Node:
        __slots__ = ('_element', '_prev', '_next')

        def __init__(self, e):
            self._element = e
            self._prev = None
            self._next = None

    def __init__(self):
        """Create a empty double linked list.
        """
        self._header = self._Node(None)
        self._tailer = self._Node(None)
        self._header._next = self._tailer
        self._tailer._prev = self._header
        self._size = 0

    def __len__(self):
        return self._size

    def __iter__(self):
        p = self._header._next
        while p is not self._tailer:
            yield p._element
            p = p._next

    def __repr__(self):
        res = ' <-> '.join(map(str, self))
        return 'header <-> ' + res + ' <-> tailer'

    def is_empty(self):
        return self._size == 0

    def _insert_between(self, e, predecessor, successor):
        """Add a node betweent two existing nodes.
        """
        node = self._Node(e)
        node._prev = predecessor
        node._next = successor
        predecessor._next = node
        successor._prev = node
        self._size += 1
        return node

    def _remove_node(self, node):
        """Remove node from the double linked list.
        """
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        res = node._element
        node._prev = node._next = node._element = None
        return res

In [15]:
class LinkedDeque(DoubleLinkedList):
    def first(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        return self._header._next._element

    def last(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        return self._tailer._prev._element

    def add_first(self, e):
        self._insert_between(e, self._header, self._header._next)

    def add_last(self, e):
        self._insert_between(e, self._tailer._prev, self._tailer)

    def del_first(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        return self._remove_node(self._header._next)

    def del_last(self):
        if self.is_empty():
            raise IndexError("Deque is empty")
        return self._remove_node(self._tailer._prev)

In [16]:
deque = LinkedDeque()
for i in range(5):
    deque.add_first(i)
for x in 'abcde':
    deque.add_last(x)
print(deque)

header <-> 4 <-> 3 <-> 2 <-> 1 <-> 0 <-> a <-> b <-> c <-> d <-> e <-> tailer


In [17]:
print(deque.del_first())
print(deque.del_first())
print(deque.del_first())
print(deque.del_last())
print(deque.del_last())
print(deque.del_last())
print(deque)

4
3
2
e
d
c
header <-> 1 <-> 0 <-> a <-> b <-> tailer


## 链表 v.s. 数组

#### 基于数组序列的优点

- 随机访问数组中任意元素的时间复杂度为O(1)
- 基于数组进行一些常数的运算效率更高
- 数组的存储结构更紧凑，更节省空间，且不会产生碎片

#### 基于链表序列的优点

- 在任意位置进行插入和删除元素的时间复杂度为O(1)
- 链表不需要一开始就分配好储存空间，也不需要考虑空间的扩容和缩容问题

# Any Questions?

## 实践项目

- 期中考试 -> 实践项目 (20%)
- 以下是可以用来实践的课题，但更希望大家能提出自己的点子，实现自己的想法：
    - 解析Verilog，DEF...
    - 抽取EDA工具的log信息
    - 分析PD项目中的数据
    - IT监控，比如磁盘监控、机器监控、LSF任务的监控
    - ......
- 开源项目，每个人都能做出贡献（不仅仅限于代码的提交）

[程序员修炼之道](https://book.douban.com/subject/5387402/) P5 《石头汤》的故事

![](img/7-8.jpg)

## 课后作业 Assignment-05

1) 课程中我们已经学习了环形链表，但在实际应用中，链表可能会存在非首尾相接的环形结构（如下图所示），请提供一种有效的算法来检测一个单向链表中是否包含环。

![](img/7-5.png)

2)反转一个单向链表，例如单向链表 4->3->2->1->0->None，反转后变成 0->1->2->3->4->None。